In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import math

In [3]:
DataSet_root = "./LIDC-IDRI"
Processed_data_dir = './data'
IMAGE_DIR = Processed_data_dir+ "/Image"
MASK_DIR = Processed_data_dir +"/Mask"
Clean_IMAGE_DIR =Processed_data_dir + '/CleanIMAGE'
Clean_MASK_DIR = Processed_data_dir +'/CleanMASK'
mask_thresh = 8

In [4]:
dataset = pd.read_csv(Processed_data_dir + '/dataset.csv')

In [4]:
dataset[dataset["is_cancer"]=='False']

,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean
36,3,0,62,./data/IMAGE/LIDC-IDRI-0003/Nodule0Slice62.png,./data/MASK/LIDC-IDRI-0003/Mask0Slice62.png,2,False,False
37,3,0,63,./data/IMAGE/LIDC-IDRI-0003/Nodule0Slice63.png,./data/MASK/LIDC-IDRI-0003/Mask0Slice63.png,2,False,False
38,3,0,64,./data/IMAGE/LIDC-IDRI-0003/Nodule0Slice64.png,./data/MASK/LIDC-IDRI-0003/Mask0Slice64.png,2,False,False
39,3,0,65,./data/IMAGE/LIDC-IDRI-0003/Nodule0Slice65.png,./data/MASK/LIDC-IDRI-0003/Mask0Slice65.png,2,False,False
40,3,0,66,./data/IMAGE/LIDC-IDRI-0003/Nodule0Slice66.png,./data/MASK/LIDC-IDRI-0003/Mask0Slice66.png,2,False,False
...,...,...,...,...,...,...,...,...
1005,67,3,172,./data/IMAGE/LIDC-IDRI-0067/Nodule3Slice172.png,./data/MASK/LIDC-IDRI-0067/Mask3Slice172.png,2,False,False
1006,67,3,173,./data/IMAGE/LIDC-IDRI-0067/Nodule3Slice173.png,./data/MASK/LIDC-IDRI-0067/Mask3Slice173.png,2,False,False
1007,67,3,174,./data/IMAGE/LIDC-IDRI-0067/Nodule3Slice174.png,./data/MASK/LIDC-IDRI-0067/Mask3Slice174.png,2,False,False
1052,69,1,52,./data/IMAGE/LIDC-IDRI-0069/Nodule1Slice52.png,./data/MASK/LIDC-IDRI-0069/Mask1Slice52.png,2,False,False


In [5]:
dataset["is_cancer"].value_counts()

True         566
Ambiguous    284
False        214
Name: is_cancer, dtype: int64

In [6]:
dataset[dataset["is_cancer"]=="Ambiguous"].count()

patient_id        284
nodule_no         284
slice_no          284
original_image    284
mask_image        284
malignancy        284
is_cancer         284
is_clean          284
dtype: int64

In [7]:
def balance_dataset(dataset):
    balance_count = min(dataset[dataset["is_cancer"]=="False"].count()[0],min(dataset[dataset["is_cancer"]=="True"].count()[0],dataset[dataset["is_cancer"]=="Ambiguous"].count()[0]))
    sampled_ds = pd.DataFrame(index=[],columns=['patient_id','nodule_no','slice_no','original_image','mask_image','malignancy','is_cancer','is_clean'])
    sampled_ds = sampled_ds.append(dataset[dataset["is_cancer"]=="Ambiguous"].sample(balance_count))
    sampled_ds = sampled_ds.append(dataset[dataset["is_cancer"]=="True"].sample(balance_count))
    sampled_ds = sampled_ds.append(dataset[dataset["is_cancer"]=="False"].sample(balance_count))
    sampled_ds = sampled_ds.reset_index()
    return sampled_ds

In [8]:

balanced_ds = balance_dataset(dataset)
balanced_ds

,index,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean
0,433,35,0,115,./data/IMAGE/LIDC-IDRI-0035/Nodule0Slice115.png,./data/MASK/LIDC-IDRI-0035/Mask0Slice115.png,3,Ambiguous,False
1,246,17,0,163,./data/IMAGE/LIDC-IDRI-0017/Nodule0Slice163.png,./data/MASK/LIDC-IDRI-0017/Mask0Slice163.png,3,Ambiguous,False
2,922,64,0,79,./data/IMAGE/LIDC-IDRI-0064/Nodule0Slice79.png,./data/MASK/LIDC-IDRI-0064/Mask0Slice79.png,3,Ambiguous,False
3,920,64,0,77,./data/IMAGE/LIDC-IDRI-0064/Nodule0Slice77.png,./data/MASK/LIDC-IDRI-0064/Mask0Slice77.png,3,Ambiguous,False
4,363,27,1,82,./data/IMAGE/LIDC-IDRI-0027/Nodule1Slice82.png,./data/MASK/LIDC-IDRI-0027/Mask1Slice82.png,3,Ambiguous,False
...,...,...,...,...,...,...,...,...,...
637,532,44,0,53,./data/IMAGE/LIDC-IDRI-0044/Nodule0Slice53.png,./data/MASK/LIDC-IDRI-0044/Mask0Slice53.png,2,False,False
638,39,3,0,65,./data/IMAGE/LIDC-IDRI-0003/Nodule0Slice65.png,./data/MASK/LIDC-IDRI-0003/Mask0Slice65.png,2,False,False
639,870,61,1,62,./data/IMAGE/LIDC-IDRI-0061/Nodule1Slice62.png,./data/MASK/LIDC-IDRI-0061/Mask1Slice62.png,2,False,False
640,708,52,0,82,./data/IMAGE/LIDC-IDRI-0052/Nodule0Slice82.png,./data/MASK/LIDC-IDRI-0052/Mask0Slice82.png,2,False,False


In [100]:
reduced_ds = balanced_ds.drop(columns=["index","patient_id","nodule_no","slice_no","malignancy","is_clean","mask_image"])


In [101]:
lbl = LabelEncoder()
enc = OneHotEncoder()

labels = lbl.fit_transform(reduced_ds["is_cancer"].values.reshape(-1,1))
reduced_ds['labels'] = labels

In [102]:
reduced_ds.reset_index(drop=True,inplace=True)

dataset = tf.data.Dataset.from_tensor_slices((reduced_ds['original_image'], reduced_ds['labels']))

In [103]:
def getImg(Img):
    img = tf.image.decode_png(Img)

    #img = img/255
    return img
def getFile(orignal_image,label):
    #img = np.load(orignal_image)
    img = tf.io.read_file(orignal_image)
    img = getImg(img)

    return img,label


In [104]:
dataset = dataset.map(getFile)

In [93]:
len_ds = reduced_ds.shape[0]
batch_size = 50
split_size = 0.8
def createTrainTest(ds):
    ds = ds.shuffle(20)
    ds = ds.batch(batch_size)
    train_ds = ds.take(math.floor(len_ds*split_size))
    test_ds = ds.skip(math.floor(len_ds*split_size)).take(math.floor(len_ds*(1-split_size)))
    return train_ds,test_ds

In [105]:
AUTOTUNE = 4
train_ds,test_ds = createTrainTest(dataset)
train_ds = train_ds.cache().prefetch(AUTOTUNE)


In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Softmax,ReLU,Flatten,MaxPooling2D

In [107]:
baseline = Sequential([
Conv2D(32,(3,3),activation = 'relu',input_shape = (512,512,1)),
Conv2D(16, 3, padding='same', activation='relu'),
 MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
 MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
 Flatten(),
  Dense(128, activation='relu'),
 Dense(3),
 
])
baseline.summary()
baseline.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics = ['acc'])
baseline.fit(train_ds,epochs=10,validation_data=test_ds)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 510, 510, 32)      320       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 510, 510, 16)      4624      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 255, 255, 16)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 255, 255, 32)      4640      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 127, 127, 64)      18496     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 63, 63, 64)       

KeyboardInterrupt: 

In [8]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17353992063779060006]